In [ ]:
pip install --upgrade soundfile librosa

     |████████████████████████████████| 1.6MB 22.1MB/s 
     |████████████████████████████████| 3.6MB 3.6MB/s 
  Created wheel for librosa: filename=librosa-0.7.1-cp36-none-any.whl size=1610157 sha256=b11cab9d9f8a3caadac550547c65ae751362cb2a61216958c954e8cb24b61361
  Stored in directory: /root/.cache/pip/wheels/07/36/47/a9a4d151332cbdaec564500af9704a0ad862cf554dcf4bfda0
Successfully built librosa
  Found existing installation: numba 0.40.1
    Uninstalling numba-0.40.1:
      Successfully uninstalled numba-0.40.1
  Found existing installation: librosa 0.6.3
    Uninstalling librosa-0.6.3:
      Successfully uninstalled librosa-0.6.3


In [ ]:
# https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
import random

y, sr = librosa.load('2342.wav', duration=5, sr=16000)
# sf.write('2330_0.wav', y, sr)
# ipd.Audio('2330_0.wav')
D = np.abs(librosa.stft(y))**2
S = librosa.feature.melspectrogram(S=D, sr=sr, n_mels=64, fmax=8000)
plt.subplot(121)
S_dB = librosa.power_to_db(S, ref=np.max)
librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', sr=sr, fmax=8000)
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-frequency spectrogram')
plt.tight_layout()
# plt.show()

y_hat = librosa.feature.inverse.mel_to_audio(S, sr)
# sf.write('2330_1.wav', y_hat, sr)
# ipd.Audio('2330_1.wav')
ipd.Audio(y_hat, rate=sr)

import torch
import torch.nn as nn
import torch.optim as optim

device = 'cuda:0'
class Encoder(nn.Module):
    def __init__(self, input_dim, enc_hid_dim, dec_hid_dim, n_layers, dropout):
        super(Encoder,self).__init__()
        self.input_dim = input_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.n_layers = n_layers
        self.rnn = nn.GRU(input_dim, enc_hid_dim, n_layers, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        # self.dropout = lambda x: x
        
    def forward(self, src):
        
        #src = [src sent len, batch size]
        
        embedded = src
        
        #embedded = [src sent len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src sent len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        #outputs = [src sent len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        # print("Values returned by encoder: ", outputs.shape, hidden.shape)
        return outputs, hidden

class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Parameter(torch.rand(dec_hid_dim))
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat encoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src sent len, dec hid dim]
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src sent len, dec hid dim]
        
        energy = energy.permute(0, 2, 1)
        
        #energy = [batch size, dec hid dim, src sent len]
        
        #v = [dec hid dim]
        
        v = self.v.repeat(batch_size, 1).unsqueeze(1)

        #v = [batch size, 1, dec hid dim]
                
        attention = torch.bmm(v, energy).squeeze(1)
        
        #attention= [batch size, src len]
        
        return torch.nn.functional.softmax(attention, dim=1)
        


class Decoder(nn.Module):
    def __init__(self, output_dim, enc_hid_dim, dec_hid_dim, n_layers, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
                
        self.rnn = nn.GRU((enc_hid_dim * 2) + output_dim, dec_hid_dim, n_layers)
        
        self.out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + output_dim, output_dim)
        
        # self.dropout = nn.Dropout(dropout)
        self.dropout = lambda x: x
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        # print("input.size in Decoder forward", input.size())
        #input = [1, batch size]
        
        embedded = input
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)

         #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [sent len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #sent len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        output = self.out(torch.cat((output, weighted, embedded), dim = 1))
        
        #output = [bsz, output dim]
        # print("Output.size() in Decoder forward: ", output.size())
        return output, hidden.squeeze(0)

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src sent len, batch size]
        #trg = [trg sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        max_len = trg.shape[0]
        # print("max_len: ", max_len)
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        # print("Input.size in Seq2Seq forward: ", input.size())
        for t in range(1, max_len):
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # print("Teacher forcing on? ", teacher_force)
            #get the highest predicted token from our predictions
            # top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else output
            # print("Input shape used for next pass to decoder: ", input.shape)

        return outputs

INPUT_DIM = 64
OUTPUT_DIM = 64
ENC_HID_DIM = 32
DEC_HID_DIM = 32
ENC_N_LAYERS = 4
DEC_N_LAYERS = 1
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_N_LAYERS, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

optimizer = optim.Adam(model.parameters())
criterion = nn.MSELoss()

x = torch.tensor(np.expand_dims(S.T, 1), dtype=torch.float)
mu = x.mean(axis=0, keepdims=True)
sig = x.std(axis=0, keepdims=True)
x = (x-mu)/sig

x = x.to(device)

model.train()
        
for i in range(1000):
    src = x
    tgt = x
    optimizer.zero_grad()
    output = model(src, tgt)   
    # print("Should be same", output.shape, tgt.shape)

    loss = criterion(output, tgt)        
    loss.backward()
    # torch.nn.utils.clip_grad_norm_(model.parameters(), 100)
    optimizer.step()
    if i%10 == 9:
        # print(torch.abs(x[1]-output[1]))
        print(i, loss.item())

model.eval()
with torch.no_grad():
    output = model(x, x, 0) #turn off teacher forcing
    loss = criterion(x, x)

plt.subplot(122)
output = output.detach().cpu()*sig + mu
S_dB = librosa.power_to_db(output[:,0,:].T, ref=np.max)
librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', sr=sr, fmax=8000)
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-frequency spectrogram')
plt.tight_layout()
plt.show()

y_hat = librosa.feature.inverse.mel_to_audio(output.numpy()[:,0,:].T, sr)
sf.write('2330_2.wav', y_hat, sr)
# ipd.Audio('2330_2.wav')
ipd.Audio(y_hat, rate=sr)